In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import box

In [ ]:
layer1_gdf = gpd.read_file(r"D:\layer1_fixgeom.shp")
layer2_gdf = gpd.read_file(r"D:\layer2_fixgeom.shp")

print(len(layer1_gdf))
print(len(layer2_gdf))

`explode()`

In [ ]:
layer1_gdf  = layer1_gdf.explode(ignore_index=False)
layer2_gdf = layer2_gdf.explode(ignore_index=False)

`drop_duplicates()`

In [ ]:
layer1_gdf  = layer1_gdf.drop_duplicates(subset='geometry')
layer2_gdf = layer2_gdf.drop_duplicates(subset='geometry')

`is_valid`

In [ ]:
layer1_gdf = layer1_gdf[layer1_gdf.is_valid]
layer2_gdf = layer2_gdf[layer2_gdf.is_valid]

In [ ]:
print(len(layer1_gdf))
print(len(layer2_gdf))

`geom_type.isin`

In [ ]:
polygon_mask = layer1_gdf['geometry'].geom_type.isin(['Polygon'])
polygon_mask.value_counts()

In [ ]:
polygon_mask = layer2_gdf['geometry'].geom_type.isin(['Polygon'])
polygon_mask.value_counts()

**grid**

`.total_bounds`

In [ ]:
layer1_gdf.total_bounds

In [ ]:
layer2_gdf.total_bounds

`box()`

*x-y bounding box is a (minx, miny, maxx, maxy) tuple.*

In [ ]:
cell_size = 100

xmin, ymin, xmax, ymax = layer2_gdf.total_bounds

rows = np.arange(ymin, ymax, cell_size)
cols = np.arange(xmin, xmax, cell_size)

polygons = [box(x, y, x + cell_size, y + cell_size) for x in cols for y in rows]

grid = gpd.GeoDataFrame({'geometry': polygons}, crs=layer2_gdf.crs)
grid

#grid.to_file(r"D:\grid.shp")


In [ ]:
print(layer1_gdf['geometry'].geom_type.value_counts())
print(layer2_gdf['geometry'].geom_type.value_counts())

`overlay()`

In [ ]:
layer1_gdf = gpd.overlay(layer1_gdf, grid, how='intersection', keep_geom_type=True)
layer2_gdf = gpd.overlay(layer2_gdf, grid, how='intersection', keep_geom_type=True)

In [ ]:
layer1_gdf['SHAPE_Area'] = layer1_gdf['SHAPE_Area'].round(2).astype(str)
layer2_gdf['SHAPE_Area'] = layer2_gdf['SHAPE_Area'].round(2).astype(str)

In [ ]:
layer1_gdf.to_file(r"D:\dev\compareZones\layer1_grid.shp")
layer2_gdf.to_file(r"D:\dev\compareZones\layer2_grid.shp")

**fixgeom in QGIS**

In [ ]:
layer1_gdf = gpd.read_file(r"D:\dev\compareZones\layer1_grid_fixgeom.shp")
layer2_gdf = gpd.read_file(r"D:\dev\compareZones\layer1_grid_fixgeom.shp")

print(len(layer1_gdf))
print(len(layer2_gdf))

`to_dict()`

In [ ]:
layer1_dict = layer1_gdf.set_index('attr1')['attr2'].to_dict()
layer1_dict


In [ ]:
layer2_dict = layer2_gdf.set_index('attr1')['attr2'].to_dict()
layer2_dict


**get values from additinal excel table**

In [ ]:
layer1_values = pd.read_excel(r"D:\table1.xlsx", sheet_name=0)
layer1_values

In [ ]:
layer2_values = pd.read_excel(r"D:\table1.xlsx", sheet_name=1)
layer2_values

In [ ]:
layer1_values_dict = layer1_values.set_index('attr3')['attr4'].to_dict()
layer1_values_dict


In [ ]:
layer2_values_dict = layer2_values.set_index('attr3')['attr4'].to_dict()
layer2_values_dict


**matrix**

In [ ]:
matrix = pd.DataFrame(index=layer2_gdf['attr2'].unique(), columns=layer1_gdf['attr2'].unique())
matrix.head(5)

In [ ]:
matrix_renamed = matrix.rename(index=layer2_values_dict, columns=layer1_values_dict)
matrix_renamed.head(5)

In [ ]:
matrix_renamed.to_excel(r"D:\matrix_renamed.xlsx")

In [ ]:
matrix_ranked = pd.read_excel(r"D:\matrix_ranked.xlsx", index_col=0)
matrix_ranked.head(5)

In [ ]:
layer1_values_dict_invert = layer1_values.set_index('attr4')['attr3'].to_dict()
layer2_values_dict_invert = layer2_values.set_index('attr4')['attr3'].to_dict()


In [ ]:
matrix_ranked_renamed = matrix_ranked.rename(index=layer2_values_dict_invert, columns=layer1_values_dict_invert)
matrix = matrix_ranked_renamed.drop(columns = ['Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'], axis=1)
matrix

**matrix to dictionary**

In [ ]:
matrix.index = matrix.index.astype(str)
matrix.columns = matrix.columns.astype(str)

In [ ]:
combination_dict = {}

for layer1 in matrix.columns:
    for layer2 in matrix.index:
         value = int(matrix.loc[layer2, layer1])

         combination_dict[(layer1, layer2)] = value

combination_dict

**intersection analysis**

In [ ]:
results = []

attribute = 'attr2'

fz_layer2_sindex = layer2_gdf.sindex

for feature_layer1 in layer1_gdf.itertuples():
    if feature_layer1.geometry and feature_layer1.geometry.is_valid:
        possible_matches_idx = list(fz_layer2_sindex.intersection(feature_layer1.geometry.bounds))
        possible_matches = layer2_gdf.iloc[possible_matches_idx]

        for feature_layer2 in possible_matches.itertuples():
            if feature_layer2.geometry and feature_layer2.geometry.is_valid:
                intersection = feature_layer1.geometry.intersection(feature_layer2.geometry)
                if intersection and not intersection.is_empty:
                    # Генерация ключа для поиска в словаре combination_dict
                    key = (str(getattr(feature_layer1, attribute)), str(getattr(feature_layer2, attribute)))

                    # Получаем результат из combination_dict
                    result = combination_dict.get(key, None)

                    # Добавляем результат в список
                    results.append({
                        "geometry": intersection,
                        "layer1_id": feature_layer1.Index,
                        "lay1_att1": getattr(feature_layer1, attribute),
                        "lay1_att2": getattr(feature_layer1, 'attr1'),       
                        "layer2_id": feature_layer2.Index,
                        "lay2_att1": getattr(feature_layer2, attribute),
                        "lay2_att2": getattr(feature_layer2, 'attr1'),
                        "result": result
                    })

# Создаём GeoDataFrame с результатами
results_gdf = gpd.GeoDataFrame(results, crs=layer1_gdf.crs)

In [ ]:
results_gdf['result']

In [ ]:
results_gdf.geom_type.value_counts()

In [ ]:
results_polygon_gdf = results_gdf[results_gdf.geom_type.isin(["Polygon", "MultiPolygon", "GeometryCollection"])]
results_polygon_gdf

In [ ]:
results_polygon_gdf.to_file(r"D:\dev\compareZones\compareZones4.3.gpkg")